In [28]:
import pandas as pd
import numpy as np

### About the Dataset
This dataset presents number of deaths and age-adjusted death rates for the 10 leading causes of death, as well as all causes of death combined, in the United States and by state for 1999-2017.  

In [29]:
cdc_original = pd.read_excel("Data/cdc_cause_of_death.xlsx")

In [30]:
cdc_original.head()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
0,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,United States,169936,49.4
1,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8
2,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7
3,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2
4,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8


In [31]:
#display the column names in the dataset
cdc_original.columns

Index(['Year', '113 Cause Name', 'Cause Name', 'State', 'Deaths',
       'Age-adjusted Death Rate'],
      dtype='object')

In [32]:
#extract required columns
cdc_data = cdc_original[['Year', 'Cause Name', 'State', 'Deaths']]
cdc_data.head()

,Year,Cause Name,State,Deaths
0,2017,Unintentional injuries,United States,169936
1,2017,Unintentional injuries,Alabama,2703
2,2017,Unintentional injuries,Alaska,436
3,2017,Unintentional injuries,Arizona,4184
4,2017,Unintentional injuries,Arkansas,1625


In [33]:
#rename Cause Name column

cdc_data = cdc_data.rename(columns = {'Cause Name': 'Cause_Name'})

In [34]:
# display the unique Cause Names listed in the dataset
cause_name_ls = cdc_data['Cause_Name'].unique()
cause_name_ls

array(['Unintentional injuries', 'All causes', "Alzheimer's disease",
       'Stroke', 'CLRD', 'Diabetes', 'Heart disease',
       'Influenza and pneumonia', 'Suicide', 'Cancer', 'Kidney disease'],
      dtype=object)

### Explore the Total Number of Deaths by Each Disease year over year

In [35]:
#Create Dataframe to identify totals for each disease for each year.
#The total number of deaths for each disease is listed in rows where the United States is the name 
#of the state. 

disease_totals = cdc_data.loc[cdc_data['State'] == 'United States'].reset_index()
# drop original index
disease_totals.drop('index', axis = 1, inplace = True)

disease_totals.head()

,Year,Cause_Name,State,Deaths
0,2017,Unintentional injuries,United States,169936
1,2017,All causes,United States,2813503
2,2017,Alzheimer's disease,United States,121404
3,2017,Stroke,United States,146383
4,2017,CLRD,United States,160201


In [36]:
# Total deaths per year is represented with the Cause Name "All Causes".
# Delete all rows where Cause Name is "All Causes" since the goal is to compare the causes individually

disease_totals = disease_totals[disease_totals.Cause_Name != 'All causes']
disease_totals.head()

,Year,Cause_Name,State,Deaths
0,2017,Unintentional injuries,United States,169936
2,2017,Alzheimer's disease,United States,121404
3,2017,Stroke,United States,146383
4,2017,CLRD,United States,160201
5,2017,Diabetes,United States,83564


### Explore data for Chronic Diseases
In this section I reduce my dataset to only include Chronic Diseases where the outcome may be affected by access to proper health care. The following diseases include Diabetes, Heart disease and Stroke.

In [37]:
#Create Dataframe to contain only data for the Chronic Diseases being considered

chronics = ['Stroke','Diabetes', 'Heart disease']

chronic_data = cdc_data[cdc_data.Cause_Name.isin(chronics)]

In [38]:
chronic_data.head()

,Year,Cause_Name,State,Deaths
156,2017,Stroke,United States,146383
157,2017,Stroke,Alabama,2931
158,2017,Stroke,Alaska,190
159,2017,Stroke,Arizona,2681
160,2017,Stroke,Arkansas,1612


In [39]:
#drop rows that display country total for each disease

chronic_data = chronic_data[chronic_data.State != 'United States']

chronic_data.head()

,Year,Cause_Name,State,Deaths
157,2017,Stroke,Alabama,2931
158,2017,Stroke,Alaska,190
159,2017,Stroke,Arizona,2681
160,2017,Stroke,Arkansas,1612
161,2017,Stroke,California,16355


In [40]:
chronic_data.Year.unique()

array([2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007,
       2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999], dtype=int64)

In [41]:
#reduce range of years in dataset to 2007 to 2017 to show 10 year range

range_yrs = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

chronic_data = chronic_data[chronic_data.Year.isin(range_yrs)]

In [42]:
chronic_data.Year.unique()


array([2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007],
      dtype=int64)

In [43]:
#create new DataFrame to extract data for 2013 and 2016.pp means pre and post meaning 2013 and 2016
years = [2009, 2010, 2015, 2016]
chronic_pp = chronic_data[chronic_data.Year.isin(years)]
chronic_pp

,Year,Cause_Name,State,Deaths
728,2016,Stroke,Alabama,2967
729,2016,Stroke,Alaska,196
730,2016,Stroke,Arizona,2556
731,2016,Stroke,Arkansas,1643
732,2016,Stroke,California,15680
...,...,...,...,...
4935,2009,Heart disease,Virginia,13397
4936,2009,Heart disease,Washington,10561
4937,2009,Heart disease,West Virginia,5038
4938,2009,Heart disease,Wisconsin,10834


In [45]:
#export cleaned datasets to csv 

chronic_pp.to_csv("Data/chronic_pp.csv", index = False)

In [46]:
chronic_data.to_csv("Data/chronic_data.csv", index = False)

In [50]:
#calculate change in disease outcomes.
#Combine 2009 + 2010 and 2015 + 2016. Subtract the two totals to calculate fields

#first pivot the data to make each year a column

chronic_pivot = chronic_pp.pivot_table('Deaths', ['Cause_Name', 'State'], columns='Year')

In [51]:
chronic_pivot.head()


Year                   2009  2010  2015  2016
Cause_Name State                             
Diabetes   Alabama     1259  1302  1255  1183
           Alaska        84    86   142   124
           Arizona     1083  1389  2081  2025
           Arkansas     890   850   886   920
           California  6979  7061  8845  9124

In [52]:
#second - create columns based on needed calculations

chronic_pivot['total_09_10'] = chronic_pivot[2009] + chronic_pivot[2010]
chronic_pivot.head()

Year                   2009  2010  2015  2016  total_09_10
Cause_Name State                                          
Diabetes   Alabama     1259  1302  1255  1183         2561
           Alaska        84    86   142   124          170
           Arizona     1083  1389  2081  2025         2472
           Arkansas     890   850   886   920         1740
           California  6979  7061  8845  9124        14040

In [67]:
chronic_pivot['total_15_16'] = chronic_pivot[2015] + chronic_pivot[2016]

chronic_pivot['deaths_change'] =  chronic_pivot['total_15_16'] - chronic_pivot['total_09_10']

chronic_pivot

Year                      2009  2010  2015  2016  total_09_10  total_15_16  \
Cause_Name State                                                             
Diabetes   Alabama        1259  1302  1255  1183         2561         2438   
           Alaska           84    86   142   124          170          266   
           Arizona        1083  1389  2081  2025         2472         4106   
           Arkansas        890   850   886   920         1740         1806   
           California     6979  7061  8845  9124        14040        17969   
...                        ...   ...   ...   ...          ...          ...   
Stroke     Virginia       3244  3293  3393  3502         6537         6895   
           Washington     2591  2548  2703  2910         5139         5613   
           West Virginia  1073  1104  1079  1039         2177         2118   
           Wisconsin      2501  2609  2618  2481         5110         5099   
           Wyoming         233   204   198   208          437          406   

Year                      deaths_change  
Cause_Name State                         
Diabetes   Alabama                 -123  
           Alaska                    96  
           Arizona                 1634  
           Arkansas                  66  
           California              3929  
...                                 ...  
Stroke     Virginia                 358  
           Washington               474  
           West Virginia            -59  
           Wisconsin                -11  
           Wyoming                  -31  

[153 rows x 7 columns]

In [69]:
#remove Cause Name as part of the index to fit data into format for Tableau
chronics = chronic_pivot.copy()

chronics = chronics.reset_index('Cause_Name')

chronics.head()



Year,Cause_Name,2009,2010,2015,2016,total_09_10,total_15_16,deaths_change
State,,,,,,,,
Alabama,Diabetes,1259,1302,1255,1183,2561,2438,-123
Alaska,Diabetes,84,86,142,124,170,266,96
Arizona,Diabetes,1083,1389,2081,2025,2472,4106,1634
Arkansas,Diabetes,890,850,886,920,1740,1806,66
California,Diabetes,6979,7061,8845,9124,14040,17969,3929


In [78]:
#extract needed data

chronic_summ = chronics[['State', 'Cause_Name', 'total_09_10', 'total_15_16', 'deaths_change']]


chronic_summ.head()


Year,State,Cause_Name,total_09_10,total_15_16,deaths_change
0,Alabama,Diabetes,2561,2438,-123
1,Alaska,Diabetes,170,266,96
2,Arizona,Diabetes,2472,4106,1634
3,Arkansas,Diabetes,1740,1806,66
4,California,Diabetes,14040,17969,3929


In [79]:
chronic_summ.to_csv('Data/chronic_summ.csv', index = False)